In It to Win IT

Token + API

In [2]:
import os

# Set the environment variable manually or use a .env file with dotenv
os.environ['TOKEN'] = 'rovjhwxiopitrddsxhrdvakheleadupk' 

# Retrieve the API token from environment variables
api_token = os.getenv('TOKEN')
api_url = 'https://codeweekend.dev:3721/api/'
files_url = 'https://codeweekend.dev:81/'

print('API TOKEN:', api_token)
print('API URL:', api_url)


API TOKEN: rovjhwxiopitrddsxhrdvakheleadupk
API URL: https://codeweekend.dev:3721/api/


API helping functions

In [3]:
import requests
import json
import time
import os

headers = {
    'authorization': f'Bearer {api_token}'
}

def show(inner_json):
    print(json.dumps(inner_json, indent=2))

def get_scoreboard():
    return requests.get(api_url + 'scoreboard', headers=headers).json()

def get_team_dashboard():
    return requests.get(api_url + 'team_dashboard', headers=headers).json()

def get_test(task_id):
    task_id_padded = '{:03d}'.format(task_id)
    url = f'{files_url}{task_id_padded}.json'
    return requests.get(url, headers=headers).content

# Returns at most 50 submissions
def get_team_submissions(offset=0, task_id=None):
    url = f'{api_url}team_submissions?offset={offset}'
    if task_id is not None:
      url += f'&task_id={task_id}'
    return requests.get(url, headers=headers).json()

def get_submission_info(submission_id, wait=False):
    url = f'{api_url}submission_info/{submission_id}'
    res = requests.get(url, headers=headers).json()
    if 'Pending' in res and wait:
        print('Submission is in Pending state, waiting...')
        time.sleep(1)
        return get_submission_info(submission_id)
    return res

# Returns submission_id
def submit(task_id, solution):
    res = requests.post(url = f'{api_url}submit/{task_id}',
                        headers=headers, files={'file': solution})
    if res.status_code == 200:
        return res.text
    print(f'Error: {res.text}')
    return None

def download_submission(submission_id):
    import urllib.request
    url = f'{api_url}download_submission/{submission_id}'
    opener = urllib.request.build_opener()
    opener.addheaders = headers.items()
    urllib.request.install_opener(opener)
    try:
        file, _ = urllib.request.urlretrieve(url, "downloaded.txt")
    except Exception as e:
        print('Failed to download submission: ', e)
        return None
    content = open(file, "r").read()
    os.remove(file)
    return content

def update_display_name(new_name):
  url = api_url + 'update_user'
  data = {
      'display_name': new_name,
      'email': "",
      'team_members': ""
  }
  return requests.post(url, json=data, headers=headers).content

# show(get_scoreboard())
# show(get_submission_info(427))
# show(get_team_dashboard())
# show(get_team_submissions())
# download_submission(476)
# get_test(1)
# update_display_name('Test 123')

Read input

In [47]:
import json

def read_heropath_input(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Example usage:
file_path = 'heropath_inputs_day1/001.json'
input_data = read_heropath_input(file_path)

Game logic

In [206]:
import json
import math

def distance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def can_move_to(x1, y1, x2, y2, speed):
    return distance(x1, y1, x2, y2) <= speed

def can_attack(hero_x, hero_y, monster_x, monster_y, attack_range):
    return distance(hero_x, hero_y, monster_x, monster_y) <= attack_range 

def find_nearest_monster(hero_x, hero_y, monsters, hero_speed, hero_range):
    def turns_to_reach(monster):
        dx = abs(monster['x'] - hero_x)
        dy = abs(monster['y'] - hero_y)
        distance_to_monster = max(dx, dy)
        # Calculate the number of turns to reach the monster
        turns = max(0, (distance_to_monster - hero_range + hero_speed - 1) // hero_speed)
        return turns

    nearest_monster = None
    min_turns = float('inf')
    max_gold = 0

    for i, monster in enumerate(monsters):
        if monster['hp'] > 1:
            turns = turns_to_reach(monster)
            if (turns < min_turns) or (turns == min_turns and monster['gold'] > max_gold):
                min_turns = turns
                max_gold = monster['gold']
                nearest_monster = i

    return nearest_monster


def calculate_xp_needed(level):
    if level == 0:
        return 1000
    return 1000 + level * 100 + (level - 1) * (level) * 50

def update_hero_stats(hero, experience_points):
    level = hero.get('level', 0)
    xp_needed = calculate_xp_needed(level)
    
    while experience_points >= xp_needed:
        experience_points -= xp_needed
        level += 1
        xp_needed = calculate_xp_needed(level)

    if hero.get('level', 0) < level:
        hero['level'] = level
        hero['speed'] = int(hero['base_speed'] * (1 + level * hero['level_speed_coeff'] / 100))
        hero['power'] = int(hero['base_power'] * (1 + level * hero['level_power_coeff'] / 100))
        hero['range'] = int(hero['base_range'] * (1 + level * hero['level_range_coeff'] / 100))

    return experience_points

In [207]:
def maximize_gold(input_data):
    hero = input_data['hero']
    hero_x = input_data['start_x']
    hero_y = input_data['start_y']
    monsters = input_data['monsters']
    num_turns = input_data['num_turns']
    width = input_data['width']
    height = input_data['height']
    
    experience_points = 0
    moves = []
    
    for turn in range(num_turns):
        update_hero_stats(hero, experience_points)
        nearest_monster_idx = find_nearest_monster(hero_x, hero_y, monsters, hero['base_speed'], hero['base_range'])
        if nearest_monster_idx is None:
            break

        monster = monsters[nearest_monster_idx]
        if can_attack(hero_x, hero_y, monster['x'], monster['y'], hero['base_range']) and monsters[nearest_monster_idx]['hp'] > 1:
            monsters[nearest_monster_idx]['hp'] -= hero['base_power'] # So that the monster is no longer found in the nearest enemy
            if monsters[nearest_monster_idx]['hp'] < 0:
                experience_points += monster['exp']
            print("=== Turn number " + str(turn))
            print(monsters[nearest_monster_idx]['hp'])
            print(hero['base_power'])
            moves.append({
                "type": "attack",
                "target_id": nearest_monster_idx
            })
           
        else:
            dx = monster['x'] - hero_x
            dy = monster['y'] - hero_y
            distance_to_monster = distance(hero_x, hero_y, monster['x'], monster['y'])
            
            if distance_to_monster <= hero['base_speed']:
                hero_x, hero_y = monster['x'], monster['y']
            else:
                move_distance = hero['base_speed']
                
                while move_distance > 0:
                    if abs(dx) > 0:
                        step_x = 1 if dx > 0 else -1
                        if (step_x ** 2) <= move_distance ** 2:
                            hero_x += step_x
                            move_distance -= abs(step_x)
                            dx -= step_x
                        else:
                            break
                    
                    if abs(dy) > 0:
                        step_y = 1 if dy > 0 else -1
                        if (step_y ** 2) <= move_distance ** 2:
                            hero_y += step_y
                            move_distance -= abs(step_y)
                            dy -= step_y
                        else:
                            break
                    
                    if (hero_x - hero['base_speed']) ** 2 + (hero_y - hero['base_speed']) ** 2 <= hero['base_speed'] ** 2:
                        move_distance = 0
                    
                    # Ensure the hero's new position is within bounds
                    hero_x = max(0, min(width, hero_x))
                    hero_y = max(0, min(height, hero_y))
                
            moves.append({
                "type": "move",
                "target_x": hero_x,
                "target_y": hero_y
            })

    return {
        "moves": moves
    }

In [193]:
file_path = 'heropath_inputs_day1/001.json'
input_data = read_heropath_input(file_path)
print(input_data)
output = maximize_gold(input_data)
# print(json.dumps(output, indent=4))

# Save to output.json
with open('output.json', 'w') as f:
    json.dump(output, f, indent=4)

{'hero': {'base_speed': 10, 'base_power': 50, 'base_range': 10, 'level_speed_coeff': 50, 'level_power_coeff': 50, 'level_range_coeff': 50}, 'start_x': 62, 'start_y': 53, 'width': 100, 'height': 100, 'num_turns': 12, 'monsters': [{'x': 30, 'y': 24, 'hp': 70, 'gold': 206, 'exp': 64}, {'x': 81, 'y': 56, 'hp': 40, 'gold': 170, 'exp': 44}, {'x': 32, 'y': 50, 'hp': 54, 'gold': 170, 'exp': 47}, {'x': 63, 'y': 46, 'hp': 29, 'gold': 141, 'exp': 29}, {'x': 50, 'y': 24, 'hp': 73, 'gold': 183, 'exp': 80}, {'x': 40, 'y': 14, 'hp': 57, 'gold': 157, 'exp': 59}, {'x': 40, 'y': 34, 'hp': 54, 'gold': 154, 'exp': 43}]}
=== Turn number 0
-21
50
=== Turn number 3
-10
50
=== Turn number 9
23
50
=== Turn number 10
-27
50


Block to create solution and submit

In [208]:
file_path = 'heropath_inputs_day1/001.json'
input_data = read_heropath_input(file_path)
print(input_data)
output = maximize_gold(input_data)
# print(json.dumps(output, indent=4))

# Save to output.json
with open('output.json', 'w') as f:
    json.dump(output, f, indent=4)
    
def example():
    with open('output.json', 'r') as file:
        solution = file.read()
    submission_id = submit(1, solution)
    print(f'Submission_id: {submission_id}')
    info = get_submission_info(submission_id, wait=True)
    print(f'Result: {info}')

example()

{'hero': {'base_speed': 10, 'base_power': 50, 'base_range': 10, 'level_speed_coeff': 50, 'level_power_coeff': 50, 'level_range_coeff': 50}, 'start_x': 62, 'start_y': 53, 'width': 100, 'height': 100, 'num_turns': 12, 'monsters': [{'x': 30, 'y': 24, 'hp': 70, 'gold': 206, 'exp': 64}, {'x': 81, 'y': 56, 'hp': 40, 'gold': 170, 'exp': 44}, {'x': 32, 'y': 50, 'hp': 54, 'gold': 170, 'exp': 47}, {'x': 63, 'y': 46, 'hp': 29, 'gold': 141, 'exp': 29}, {'x': 50, 'y': 24, 'hp': 73, 'gold': 183, 'exp': 80}, {'x': 40, 'y': 14, 'hp': 57, 'gold': 157, 'exp': 59}, {'x': 40, 'y': 34, 'hp': 54, 'gold': 154, 'exp': 43}]}
=== Turn number 0
-21
50
=== Turn number 3
-10
50
=== Turn number 9
23
50
=== Turn number 10
-27
50
Submission_id: 56428
Result: {'Ok': 494.0}


In [160]:
def process_and_submit(file_id):
    file_path = f'heropath_inputs_day1/00{file_id}.json'
    
    # Read the input data from the JSON file
    input_data = read_heropath_input(file_path)
    print(f"Input data for {file_path}:", input_data)
    
    # Process the input data to maximize gold
    output = maximize_gold(input_data)
    print(f"Output data for {file_path}:", json.dumps(output, indent=4))
    
    # Save the output to a JSON file
    output_file_path = f'output_00{file_id}.json'
    with open(output_file_path, 'w') as f:
        json.dump(output, f, indent=4)
    
    # Submit the result
    with open(output_file_path, 'r') as file:
        solution = file.read()
    submission_id = submit(file_id, solution)
    print(f'Submission_id for {file_path}: {submission_id}')
    
    # Get the submission info and print the result
    info = get_submission_info(submission_id, wait=True)
    print(f'Result for {file_path}: {info}')

def process_all_files():
    for file_id in range(1, 9):
        process_and_submit(file_id)
        time.sleep(5)

# Run the process for all files
process_all_files()

Input data for heropath_inputs_day1/001.json: {'hero': {'base_speed': 10, 'base_power': 50, 'base_range': 10, 'level_speed_coeff': 50, 'level_power_coeff': 50, 'level_range_coeff': 50}, 'start_x': 62, 'start_y': 53, 'width': 100, 'height': 100, 'num_turns': 12, 'monsters': [{'x': 30, 'y': 24, 'hp': 70, 'gold': 206, 'exp': 64}, {'x': 81, 'y': 56, 'hp': 40, 'gold': 170, 'exp': 44}, {'x': 32, 'y': 50, 'hp': 54, 'gold': 170, 'exp': 47}, {'x': 63, 'y': 46, 'hp': 29, 'gold': 141, 'exp': 29}, {'x': 50, 'y': 24, 'hp': 73, 'gold': 183, 'exp': 80}, {'x': 40, 'y': 14, 'hp': 57, 'gold': 157, 'exp': 59}, {'x': 40, 'y': 34, 'hp': 54, 'gold': 154, 'exp': 43}]}
Output data for heropath_inputs_day1/001.json: {
    "moves": [
        {
            "type": "attack",
            "target_id": 3
        },
        {
            "type": "move",
            "target_x": 69,
            "target_y": 56
        },
        {
            "type": "move",
            "target_x": 79,
            "target_y": 56
       